<a href="https://colab.research.google.com/github/thugdb/TD/blob/main/thugtorrent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/Essential-Google-Colab-Notebook/0b62cd8a437d3439a1dbece9119b73bfa890ad1a/cell_logos/Google-Drive-Logo.svg' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3><b>Mount Google Drive</b></h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [4]:
#@markdown <br><center><img src='https://www.libtorrent.org/img/logo-color.png' height="50" alt="libtorrent-logo"/></center>
#@markdown <center><h3><b>Install libtorrent</b></h3></center><br>
from IPython.display import clear_output

!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

clear_output()
print("Successfully Installed.")

Successfully Installed.


In [ ]:
import time
import os, sys, re
import libtorrent as lt
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, clear_output

savePath = "/content/drive/Shareddrives/Team Thug/qbit" #@param {type:"string"}
downloadType = "magnetLink" #@param ["torrentFile", "magnetLink"]

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

if downloadType == "torrentFile":
    source = files.upload()
    params = {
        "save_path": savePath,
        "ti": lt.torrent_info(list(source.keys())[0])
    }
    downloads.append(ses.add_torrent(params))
    clear_output()
else:
    magnetLink = input("magnetLink: ")
    params = {
        "save_path": savePath
    }
    downloads.append(lt.add_magnet_uri(ses, magnetLink, params))
    clear_output()

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [widgets.FloatSlider(step=0.01, disabled=True, layout=layout, style=style) for _ in downloads]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/Essential-Google-Colab-Notebook/0b62cd8a437d3439a1dbece9119b73bfa890ad1a/cell_logos/qBittorrent-Logo.svg' height="50" alt="qBittorrent-logo"/></center>
#@markdown <center><h3><b>Install qBittorrent</b></h3></center><br>
import os
import time
import json
import urllib.request
from IPython.display import clear_output

if os.path.isfile('/tools/node/bin/lt') is False:
    !npm install -g npm
    !npm install -g localtunnel
else:
    print("Localtunnel is already installed.")

if os.path.isfile("/usr/bin/qbittorrent-nox") is False:
    !apt update -qq -y && yes "" | add-apt-repository ppa:qbittorrent-team/qbittorrent-stable
    !apt install qbittorrent-nox
    !mkdir -p -m 666 /{content/qBittorrent,root/{.qBittorrent_temp,.config/qBittorrent}} && curl -s https://pastebin.com/raw/7TEALGNz -o /root/.config/qBittorrent/qBittorrent.conf
    print("qBittorrent successfully installed.")
    clear_output(wait=True)
else:
    print("qBittorrent is already installed.")
    clear_output(wait=True)
    !pkill qbittorrent-nox

!qbittorrent-nox -d --webui-port=5454
print("qBittorrent is started.")
clear_output(wait=True)
!lt --port 5454